## MNIST Linear

In [1]:
import tensorflow as tf
import numpy as np

/usr/lib/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)


In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.InteractiveSession()

We start building the computation graph by creating nodes for the input images and target output classes.

In [4]:
x = tf.placeholder(tf.float32,[None,784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

The input images x will consist of a 2d tensor of floating point numbers. Here we assign it a shape of [None, 784], where 784 is the dimensionality of a single flattened 28 by 28 pixel MNIST image, and None indicates that the first dimension, corresponding to the batch size, can be of any size. The target output classes y_ will also consist of a 2d tensor, where each row is a one-hot 10-dimensional vector indicating which digit class (zero through nine) the corresponding MNIST image belongs to.

We now define the weights W and biases b for our model. We initialize both W and b as tensors full of zeros

In [5]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Before Variables can be used within a session, they must be initialized using that session. This step takes the initial values (in this case tensors full of zeros) that have already been specified, and assigns them to each Variable.

In [6]:
sess.run(tf.global_variables_initializer())

We can now implement our regression model. We multiply the vectorized input images x by the weight matrix W, add the bias b

In [7]:
y = tf.matmul(x,W) + b

### Training

Our loss function is the cross-entropy between the target and the softmax activation function applied to the model's prediction.

In [9]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=y))

 For training, we will use steepest gradient descent, with a step length of 0.5, to descend the cross entropy.

In [10]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

What TensorFlow actually did in that single line was to add new operations to the computation graph. These operations included ones to compute gradients, compute parameter update steps, and apply update steps to the parameters.

The returned operation train_step, when run, will apply the gradient descent updates to the parameters. Training the model can therefore be accomplished by repeatedly running train_step.

In [11]:
for _ in range(1000):
  batch = mnist.train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

Note that you can replace any tensor in your computation graph using feed_dict -- it's not restricted to just placeholders.

### Evaluating

In [12]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

That gives us a list of booleans. To determine what fraction are correct, we cast to floating point numbers and then take the mean.

In [13]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Finally, we can evaluate our accuracy on the test data

In [14]:
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.918


### Write graph and close session

In [15]:
tf.summary.FileWriter("log/MNIST_linear/",sess.graph)

In [16]:
sess.close()